In [8]:
import pandas as pd
import os
import datetime
import numpy as np
from tqdm import tqdm

game_ids.xlsx랑 collected_data가 있어야 돌아감.

In [3]:
strcolumn_dict = {"tournamentId":"str", "matchId":"str", "gameId":"str", "patch":"str"}
temp1 = [f"esportsPlayerId_{i}" for i in range(10)]
temp2 = ["str" for i in range(10)]
esportsPlayerId_type_dict = dict(zip(temp1, temp2))
strcolumn_dict.update(esportsPlayerId_type_dict)
game_ids_df = pd.read_excel("../data/game_ids.xlsx", dtype=strcolumn_dict)
game_ids_df["startTime(match)"] = game_ids_df["startTime(match)"].apply(lambda x : datetime.datetime.strptime(x[:19], "%Y-%m-%dT%H:%M:%S"))

game_ids_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9287 entries, 0 to 9286
Data columns (total 47 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   startTime(match)         9287 non-null   datetime64[ns]
 1   tournamentId             9287 non-null   object        
 2   blockName                9287 non-null   object        
 3   leagueName               9287 non-null   object        
 4   matchId                  9287 non-null   object        
 5   bestof                   9287 non-null   int64         
 6   blueteam_name            9287 non-null   object        
 7   blueteam_code            9287 non-null   object        
 8   blueteam_win             9287 non-null   int64         
 9   redteam_name             9287 non-null   object        
 10  redteam_code             9287 non-null   object        
 11  redteam_win              9287 non-null   int64         
 12  gameNumberInAMatch       9287 non-

In [4]:
game_ids_with_picks_df = game_ids_df[pd.isna(game_ids_df[f"pick_0"]) == False]
game_ids_with_picks_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8023 entries, 0 to 9286
Data columns (total 47 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   startTime(match)         8023 non-null   datetime64[ns]
 1   tournamentId             8023 non-null   object        
 2   blockName                8023 non-null   object        
 3   leagueName               8023 non-null   object        
 4   matchId                  8023 non-null   object        
 5   bestof                   8023 non-null   int64         
 6   blueteam_name            8023 non-null   object        
 7   blueteam_code            8023 non-null   object        
 8   blueteam_win             8023 non-null   int64         
 9   redteam_name             8023 non-null   object        
 10  redteam_code             8023 non-null   object        
 11  redteam_win              8023 non-null   int64         
 12  gameNumberInAMatch       8023 non-null 

In [5]:
PARTICIPANT_NUMBER_OF_A_TEAM = 5
participant_ids_by_role = []
i = 0
for i in range(PARTICIPANT_NUMBER_OF_A_TEAM):       # 탑 정글 미드 원딜 서폿 이렇게..
    game_ids_with_playerId_blue = game_ids_with_picks_df[pd.isna(game_ids_with_picks_df[f"esportsPlayerId_{i}"]) == False]                                  # 블루팀(0~5)에서 서머너ID가 있는 것만
    game_ids_with_playerId_red = game_ids_with_picks_df[pd.isna(game_ids_with_picks_df[f"esportsPlayerId_{i + PARTICIPANT_NUMBER_OF_A_TEAM}"]) == False]    # 레드팀(5~9)에서 서머너ID가 있는 것만
    participant_ids_by_role.append(list(set(pd.concat([game_ids_with_playerId_blue[f"esportsPlayerId_{i}"],game_ids_with_playerId_red[f"esportsPlayerId_{i + PARTICIPANT_NUMBER_OF_A_TEAM}"]], ignore_index=True))))

print([len(i) for i in participant_ids_by_role])
print(participant_ids_by_role[0])


[346, 370, 368, 355, 394]
['101383792831678607', '104573231995499518', '98767991787052097', '107569568015156338', '101799086969700514', '106466536685100550', '105957589636391222', '107705430994631113', '101383792855140515', '106105043525540824', '104251865849200240', '105709413113050046', '107614697432935001', '99566408542840078', '101796918394601598', '107568579494407911', '99566404541748447', '98767975938371599', '109675627129279585', '98926509821954141', '99566408210057665', '109675500511180816', '107568586413146262', '105515229738531937', '111720165230650519', '107688960045460307', '100725844987233506', '109828407662355333', '98767975933568904', '107251528271309568', '101388913291808185', '105581684606195509', '107647477264255840', '100164952147744283', '104266795407626462', '108316132748778567', '108553450969769750', '107705432370779918', '99566408307919824', '107251646222113259', '100131567589299245', '105709428644866635', '111934286166932665', '107635905118503535', '107642475024

아 이제 각 포지션별 플레이어ID 리스트 뽑았고...

In [6]:
game_ids_df_sorted = game_ids_with_picks_df.sort_values(by="startTime(match)", ascending=False) # 최근 경기가 위로 오도록 내림차순 정렬
game_ids_df_sorted.head()

,startTime(match),tournamentId,blockName,leagueName,matchId,bestof,blueteam_name,blueteam_code,blueteam_win,redteam_name,...,esportsPlayerId_0,esportsPlayerId_1,esportsPlayerId_2,esportsPlayerId_3,esportsPlayerId_4,esportsPlayerId_5,esportsPlayerId_6,esportsPlayerId_7,esportsPlayerId_8,esportsPlayerId_9
4059,2024-05-28 10:00:00,112439909522772195,Week 1,LCO,112439909523296569,2,ION Global Esports,ION,2,Kanga Esports,...,111720042622049047,112477437508207101,111720045287922459,111720027901143523,111720051264281664,110462937178590834,111720122152475827,112477352770135560,112477356024384776,112477373979171280
4058,2024-05-28 10:00:00,112439909522772195,Week 1,LCO,112439909523296569,2,ION Global Esports,ION,2,Kanga Esports,...,111720042622049047,112477437508207101,111720045287922459,111720027901143523,111720051264281664,110462937178590834,111720122152475827,112477352770135560,112477356024384776,112477373979171280
4057,2024-05-28 08:00:00,112439909522772195,Week 1,LCO,112439909523296494,2,Fury,FRY,0,Team Bliss,...,105647956818593679,111720087560530528,110462943397555237,108604888219092306,111758035988327173,100160799381721105,104668539132702977,101383792849832093,103643243250697054,101383792831678607
4056,2024-05-28 08:00:00,112439909522772195,Week 1,LCO,112439909523296494,2,Fury,FRY,0,Team Bliss,...,105647956818593679,111720087560530528,110462943397555237,108604888219092306,111758035988327173,100160799381721105,104668539132702977,101383792849832093,103643243250697054,101383792831678607
4054,2024-05-27 10:00:00,112439909522772195,Week 1,LCO,112439909523296515,2,Antic Esports,ANC,2,Dire Wolves,...,107634897325128022,107634941727734818,105709436604717285,99566406296347493,102546029192465275,107647480732814180,112489008809745004,112488998397516384,102206314367722668,105714276869190870


In [9]:
DETAIL_PATH = "../data/collected_data/"
file_list_in_detail = os.listdir(DETAIL_PATH)
for i, participant_ids in enumerate(participant_ids_by_role):     # 탑 정글 미드 원딜 서폿에 대해
    for participant_id in tqdm(participant_ids, total = len(participant_ids)):  # 모든 플레이어들에 대해
        filtered_df = game_ids_df_sorted[game_ids_df_sorted[f"esportsPlayerId_{i}"] == participant_id].head(5)    # 제일 최근 경기 5개를 가져온다.
        for j, row in filtered_df.iterrows():       # 와 3중 반복문 이게 맞나...?
            if row["gameId"] + ".xlsx" in file_list_in_detail:     # invalid말고 온전히 남아 있으면
                detail_df = pd.read_excel(DETAIL_PATH + row["gameId"] + ".xlsx")   # 데이터를 읽어와서
                
                
        

 17%|█▋        | 58/346 [04:12<20:51,  4.35s/it]


KeyboardInterrupt: 

이제 각 파일 들어가서 원하는 자료 뽑는 함수 만들어서 저기다 넣으면 됨...